## Import packages and create postgres connection

In [1]:
%load_ext sql

In [38]:
import pandas as pd
import os
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine
import networkx as nx
from bokeh.io import output_notebook, show, save
output_notebook()

from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx

POSTGRES_USER = os.environ['POSTGRES_USER']
POSTGRES_PASSWORD = os.environ['POSTGRES_PASSWORD']
POSTGRES_ADDRESS = os.environ['POSTGRES_ADDRESS']
POSTGRES_PORT = os.environ['POSTGRES_PORT']
POSTGRES_DB = os.environ['POSTGRES_DB']
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
    username=POSTGRES_USER,
    password=POSTGRES_PASSWORD,
    ipaddress=POSTGRES_ADDRESS,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DB))

Loading BokehJS ...

In [3]:
%sql $postgres_str

'Connected: app_user@app_db'

In [4]:
cnx = create_engine(postgres_str)

# Part 1: Create data model
</br>
<img src="./data_model.png">

## Explanation
</br>
The given data and problem set points towards a graph database as a good candidate for a data storage solution. However, i don't have too much experience with graph databases. Therefore, I will be using a postgres database and will model the data in 3NF (Third Normal Form). 

In [5]:
%sql DROP SCHEMA public CASCADE; CREATE SCHEMA public;

 * postgresql://app_user:***@db:5432/app_db
Done.
Done.


[]

In [6]:
%sql create table if not exists person ( id text primary key, name text not null, phone text);

 * postgresql://app_user:***@db:5432/app_db
Done.


[]

In [7]:
%sql create table if not exists car ( id text primary key, make text not null, model text not null);

 * postgresql://app_user:***@db:5432/app_db
Done.


[]

In [8]:
%sql create table if not exists team (id text primary key, name text not null);

 * postgresql://app_user:***@db:5432/app_db
Done.


[]

In [9]:
%sql create table if not exists district (id text primary key, name text not null);

 * postgresql://app_user:***@db:5432/app_db
Done.


[]

In [10]:
%sql create table if not exists school (id text primary key, name text not null, district_id text, foreign key (district_id) references district (id));

 * postgresql://app_user:***@db:5432/app_db
Done.


[]

In [11]:
%sql create table if not exists attends (person_id text not null, school_id text not null, foreign key (person_id) references person (id), foreign key (school_id) references school (id));

 * postgresql://app_user:***@db:5432/app_db
Done.


[]

In [12]:
%sql create table if not exists drives (person_id text not null, car_id text not null, foreign key (person_id) references person (id), foreign key (car_id) references car (id));

 * postgresql://app_user:***@db:5432/app_db
Done.


[]

In [13]:
%sql create table if not exists plays_for (person_id text not null, team_id text not null, foreign key (person_id) references person (id), foreign key (team_id) references team (id));

 * postgresql://app_user:***@db:5432/app_db
Done.


[]

In [14]:
%sql create table if not exists studies_with (person_id text not null, person2_id text not null, foreign key (person_id) references person (id), foreign key (person2_id) references person (id));

 * postgresql://app_user:***@db:5432/app_db
Done.


[]

In [15]:
%sql create table if not exists friends_with (person_id text not null, person2_id text not null, foreign key (person_id) references person (id), foreign key (person2_id) references person (id));

 * postgresql://app_user:***@db:5432/app_db
Done.


[]

## Read data and save to DB

In [16]:
# Read data
entity_properties = pd.read_csv('./datasets/entity_properties.txt', sep='\t')
relationships = pd.read_csv('./datasets/entity_relationships.txt', sep='\t')

# Clean dataframes
entity_properties.columns = [x.strip() for x in entity_properties.columns]
relationships.columns = [x.strip() for x in relationships.columns]
for col in entity_properties.columns:
    entity_properties = entity_properties.apply(lambda x: x.str.strip())
for col in relationships.columns:
    relationships = relationships.apply(lambda x: x.str.strip())

In [17]:
# store unique entities
entities = set(relationships['Type1'].unique()) | set(relationships['Type2'].unique())
for ent in entities:
    tmp1 = set(relationships.loc[relationships['Type1'] == ent]['ID1'].unique())
    tmp2 = set(relationships.loc[relationships['Type2'] == ent]['ID2'].unique())
    ent_ids = list(tmp1 | tmp2)
    if ent == 'Car':
        makes = entity_properties.loc[entity_properties['ID'].isin(ent_ids) & (entity_properties['Property'] == 'Make')][['ID', 'Value']]
        makes.columns = ['id', 'make']
        models = entity_properties.loc[entity_properties['ID'].isin(ent_ids) & (entity_properties['Property'] == 'Model')][['ID', 'Value']]
        models.columns = ['id', 'model']
        cars = makes.merge(models, how='left', on='id')
        cars.to_sql(ent.lower(), cnx, if_exists='append', index=False)
    else:
        datadf = entity_properties.loc[entity_properties['ID'].isin(ent_ids) & (entity_properties['Property'] == 'Name')][['ID', 'Value']].copy()
        datadf.columns = ['id', 'name']
        datadf.to_sql(ent.lower(), cnx, if_exists='append', index=False)

In [18]:
# store relationships
unique_rels = relationships['Relationship'].unique().tolist()
for rel in unique_rels:
    tmp = relationships.loc[relationships['Relationship'] == rel][['ID1', 'ID2']].copy()
    table = rel.lower()
    if rel == 'STUDIES_WITH' or rel == 'FRIENDS_WITH':
        tmp.columns = [f'person_id', f'person2_id']
    elif rel == 'ATTENDS':
        tmp.columns = [f'person_id', f'school_id']
    elif rel == 'DRIVES':
        tmp.columns = [f'person_id', f'car_id']
    elif rel == 'PLAYS_FOR':
        tmp.columns = [f'person_id', f'team_id']
    elif rel == 'BELONGS_TO':
        tmp.columns = ['id', f'district_id']
        schools = pd.read_sql_table('school', cnx)[['id', 'name']].copy()
        tmp = tmp.merge(schools, how='left', on='id')
        table = 'school'
    tmp.to_sql(table, cnx, if_exists='replace', index=False)

Verify tables were created correctly

In [19]:
%sql select person.id as person_id, person.name as person_name, attends.school_id, school.name as school_name, district.name as district_name from person left join attends on attends.person_id = person.id left join school on school.id = attends.school_id left join district on school.district_id = district.id

 * postgresql://app_user:***@db:5432/app_db
11 rows affected.


person_id,person_name,school_id,school_name,district_name
A,Ally,X,Shermer,Shermer
B,Molly,X,Shermer,Shermer
I,Brad,None,None,None
H,Martin,None,None,None
G,Kieff,None,None,None
J,Joe,None,None,None
Z,Anthony,None,None,None
C,Emilio,None,None,None
K,Kevin,None,None,None
L,Demm,None,None,None


# Part 2: Finding cliques in data
## Explanation:
Initially, I only included the edges of people who were **friends** **with** each other (as per the description of the problem). That graph resulted in only two person cliques. In order to make it more interesting, I added the connection[s] of people who **studied** **with** each other as well. Unfortunately, this only added one more edge to the graph. Fortunately, that one more edge created a clique of three vertices as you will see in the last cell.

In [56]:
friends = pd.read_sql("select person.id as person_id, person2.id as person2_id from friends_with fw left join person on person.id = fw.person_id left join person person2 on person2.id = fw.person2_id", cnx)
studybuddy = pd.read_sql("select p1.id as person_id, p2.id as person2_id from studies_with sw join person p1 on p1.id = sw.person_id join person p2 on p2.id = sw.person2_id", cnx)
edges = pd.concat([friends, studybuddy])
G = nx.from_pandas_edgelist(edges, source='person_id', target='person2_id')


In [55]:
title = "Friends and Study Buddies"
HOVER_TOOLTIPS = [("Person", "@index")]
#Create a plot — set dimensions, toolbar, and title
plot = figure(
    tooltips=HOVER_TOOLTIPS,
    tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
    x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object with spring layout
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(G, nx.spring_layout, scale=10, center=(0, 0))

#Set node size and color
network_graph.node_renderer.glyph = Circle(size=15, fill_color='skyblue')

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

#Add network graph to the plot
plot.renderers.append(network_graph)

show(plot)

In [57]:
print('List of cliques:')
for c in nx.algorithms.clique.find_cliques(G):
    print(c)

List of cliques:
['C', 'G']
['C', 'A', 'B']
['K', 'L']
['K', 'H']
['K', 'J']
['K', 'I']
['A', 'J']
['Z', 'I']
['G', 'H']
['D', 'L']
